In [2]:
import csv
import base64
import time
import fnmatch
import urllib
import traceback
from datetime import datetime, timedelta
from github import Github, StatsContributor
from awesomecure import awesome2py
from urllib.parse import urlparse
from os import getenv, path
from dotenv import load_dotenv
from termcolor import colored

In [3]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = "{:02d}:{:02d}".format(mins, secs)
        print(timeformat, end="\r")
        time.sleep(1)
        t -= 1
    print("\n\n\n\n\n")

In [4]:
load_dotenv()
g = Github(getenv("GITHUB"))

In [5]:
awesome_url = "https://github.com/protontypes/open-sustainable-technology"
awesome_path = urlparse(awesome_url).path.strip("/")
filename = "README.md"

awesome_repo = g.get_repo(awesome_path)
awesome_content_encoded = awesome_repo.get_contents(
    urllib.parse.quote(filename)
).content
awesome_content = base64.b64decode(awesome_content_encoded)

filehandle = open(".awesome.md", "w")
filehandle.write(awesome_content.decode("utf-8"))
filehandle.close()
repo_dict = awesome2py.AwesomeList(".awesome.md")

print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Appropedia - Shares knowledge to build rich, sustainable lives. [https://www.appropedia.org/Welcome_to_Appropedia] - Awesome Clean Tech - Clean technology is any process, product, or service that reduces negative environmental impacts through significant energy efficiency improvements, the sustainable use of resources, or environmental protection activities. [https://github.com/nglgzz/awesome-clean-tech] - Awesome Open Climate Science - Awesome Open Atmospheric, Ocean, and Climate Science. [https://github.com/pangeo-data/awesome-open-climate-science] - Awesome Earth - "What can I do about the climate crisis?" Here are 326 things you can do. [https://github.com/philsturgeon/awesome-earth] - Awesome Open Geoscience - Curated from repositories that make our lives as geoscientists, hackers and data wranglers easier or just more awesome. [https://github.com/softwareunderground/awesome-open-geoscience] - Datasets APIs and open source projects related to Climate Change - A curated list of 

In [6]:
csv_github_projects = open("./csv/github_projects.csv", "w", newline="")
csv_github_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "topics",
    "rubric",
    "last_update",
    "stargazers_count",
    "project_active_checkbox",
    "dominating_language",
    "organization",
    "organization_user_name",
    "languages",
    "homepage",
    "created",
    "license",
    "total_commits_last_year",
    "total_number_of_commits",
    "last_issue_closed",
    "open_issues",
    "closed_pullrequests",
    "closed_issues",
    "open_pullrequests",
    "last_released_date",
    "last_release_tag_name",
    "community_development_distribution_score",
    "good_first_issue",
    "contributors",
    "accepts_donations",
    "donation_platforms",
    "code_of_conduct",
    "contribution_guide",
    "organization_avatar"
]
writer_github_projects = csv.DictWriter(
    csv_github_projects, fieldnames=csv_github_fieldnames
)
writer_github_projects.writeheader()

csv_github_organizations = open("./csv/github_organizations.csv", "w", newline="")
csv_github_organizations_fieldnames = [
    "organization_name",
    "organization_user_name",
    "organization_url",
    "organization_location",
    "organization_avatar",
    "organization_public_repos",
    "organization_created",
    "organization_last_update",
]

                            
writer_github_organizations = csv.DictWriter(
    csv_github_organizations, fieldnames=csv_github_organizations_fieldnames
)
writer_github_organizations.writeheader()
organization_list = []

csv_gitlab_projects = open("./csv/gitlab_projects.csv", "w", newline="")
csv_gitlab_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "rubric",
]
writer_gitlab_projects = csv.DictWriter(
    csv_gitlab_projects, fieldnames=csv_gitlab_fieldnames
)
writer_gitlab_projects.writeheader()

csv_custom_platform_projects = open(
    "./csv/custom_platform_projects.csv", "w", newline=""
)
csv_custom_platform_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "rubric",
]
writer_custom_platform_projects = csv.DictWriter(
    csv_custom_platform_projects, fieldnames=csv_custom_platform_fieldnames
)
writer_custom_platform_projects.writeheader()


for r in repo_dict.rubrics:
    for entry in r.entries:
        print("------------------------")
        print("Processing %s" % entry.name)
        print("URL: %s" % entry.url)
        if urlparse(entry.url).netloc == "github.com":
            print("%s is a GitHub project" % entry.name)
            try:

                remaining, limit = g.rate_limiting
                resettime = g.rate_limiting_resettime
                min_requests = 200
                if remaining < min_requests:
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = datetime.now().timestamp()
                    countdown((int(resettime) - int(current_time) + 2))

                print(
                    "GitHub Requests | Limit: {}, Remaining: {}".format(
                        limit, remaining
                    )
                )
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(entry.url).path.strip("/")
                user, project_name = path.split(repo_path)
                repo = g.get_repo(repo_path)
                contents_root = repo.get_contents("")
                releases = repo.get_releases()
                commits = repo.get_commits()
                
                # Check if the project is still active
                closed_issues = repo.get_issues(state="closed")
                open_issues = repo.get_issues(state="open")

                closed_pullrequests = repo.get_pulls(state="closed")
                open_pullrequests = repo.get_pulls(state="open")

                ## preprocessing
                if closed_issues.totalCount > 0:
                    last_issue_closed = closed_issues[0].updated_at

                total_commits_last_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_last_year = total_commits_last_year + event.total

                last_issue_closed_timedelta = datetime.utcnow() - last_issue_closed

                # Do not process inactive projects
                if (
                    last_issue_closed_timedelta > timedelta(days=365)
                    and total_commits_last_year == 0
                    or repo.archived
                ):
                    print("%s is an inactive project" % entry.name)
                    project_active_checkbox = "false"
                else:
                    print("%s is an active project" % entry.name)
                    project_active_checkbox = "true"

                try:
                    license = repo.get_license()
                    if license.license.spdx_id == "NOASSERTION":
                        print("Custom license found")
                        license_name = "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print("No license information found")
                    license_name == "UNDEFINED"

                labels = ",".join([entry.name for entry in repo.get_labels()])
                topics = ",".join(repo.get_topics())

                languages_states = repo.get_languages()
                programming_languages = ",".join(languages_states.keys())
                
                try:
                    dominating_language = list(languages_states.keys())[0]
                except:
                    dominating_language = ""

                try:
                    dotfolder_file = repo.get_contents(".github")
                    for file in dotfolder_file:
                        if file.path == ".github/FUNDING.yml":
                            print("Funding found")
                            donation_platforms = base64.b64decode(file.content)
                            accepts_donations = "true"
                        else:
                            donation_platforms = None
                            accepts_donations = "false"
                except:
                    print("No funding information found")
                    donation_platforms = None
                    accepts_donations = "false"

                try:
                    code_of_conduct = "false"
                    contribution_guide = "false"
                    for file_content in contents_root:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                    for file_content in dotfolder_file:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                except Exception as e:
                    print(e)

                contributors = repo.get_stats_contributors()
                contributor_activity = {}
                commits_total = 0
                for individuum in contributors:
                    contributor_activity[individuum.author.login] = individuum.total
                    commits_total = commits_total + individuum.total

                sorted_contributor = dict(
                    sorted(contributor_activity.items(), key=lambda item: item[1])
                )
                weighted_contribution = {
                    k: v / commits_total for k, v in sorted_contributor.items()
                }

                # Create a very simple community health score that shows how much the project is focused on one developer
                community_development_distribution_score = 1 - max(
                    weighted_contribution.values()
                )

                try:
                    last_released_date = releases[0].published_at.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    )
                    last_release_tag_name = releases[0].tag_name

                except Exception as e:
                    print("No Release found")
                    last_released_date = ""
                    last_release_tag_name = ""
                    print(e)

                total_number_of_commits = commits.totalCount

                # Gathering organization data
                if repo.organization is None:
                    organization_user_name = None

                elif repo.organization.login not in organization_list:
                    organization_user_name = repo.organization.login
                    organization_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    organization_public_repos = repo.organization.public_repos
                    organization_location = repo.organization.location
                    organization_url = repo.organization.html_url
                    organization_created = repo.organization.created_at.strftime("%Y/%m/%d, %H:%M:%S")
                    organization_repos = g.search_repositories(query='user:'+organization_user_name, sort='updated')     
                    organization_last_update = organization_repos[0].updated_at.strftime("%Y/%m/%d, %H:%M:%S")
                        
                    print("Last Update")
                    print(organization_repos[0])
                        
                    organization_data = {
                        "organization_name": organization_name,
                        "organization_user_name":organization_user_name,
                        "organization_url":organization_url,
                        "organization_avatar": organization_avatar,
                        "organization_location": organization_location,
                        "organization_public_repos": organization_public_repos,
                        "organization_created": organization_created,
                        "organization_last_update": organization_last_update
                    }

                    organization_list.append(organization_user_name)
                    print("Organization Data:")
                    print(organization_data)
                    writer_github_organizations.writerow(organization_data)

                else:
                    organization_name = repo.organization.login
                    organization_user_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    

                project_data = {
                    "project_name": entry.name,
                    "git_namespace": user,
                    "git_url": repo.clone_url,
                    "rubric": r.key,
                    "oneliner": entry.text[2:],
                    "topics": topics,
                    "organization": organization_name,
                    "organization_user_name": organization_user_name,
                    "created": repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "last_update": repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "total_commits_last_year": total_commits_last_year,
                    "project_active_checkbox": project_active_checkbox,
                    "last_issue_closed": last_issue_closed.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    ),
                    "last_released_date": last_released_date,
                    "last_release_tag_name": last_release_tag_name,
                    "total_number_of_commits": total_number_of_commits,
                    "community_development_distribution_score": community_development_distribution_score,
                    "stargazers_count": repo.stargazers_count,
                    "dominating_language": dominating_language,
                    "languages": programming_languages,
                    "homepage": repo.homepage,
                    "closed_issues": closed_issues.totalCount,
                    "open_issues": open_issues.totalCount,
                    "closed_pullrequests": closed_pullrequests.totalCount,
                    "open_pullrequests": open_pullrequests.totalCount,
                    "good_first_issue": repo.get_issues(
                        state="open", labels=["good first issue"]
                    ).totalCount,
                    "license": license_name,
                    "contributors": repo.get_contributors().totalCount,
                    "accepts_donations": accepts_donations,
                    "donation_platforms": donation_platforms,
                    "code_of_conduct": code_of_conduct,
                    "contribution_guide": contribution_guide,
                    "organization_avatar":organization_avatar
                }

                print("Project Data:")
                print(project_data)
                writer_github_projects.writerow(project_data)

            except Exception as e:
                print(colored("Failed to gather project information:"))
                print(colored(e, "red"))
                print(traceback.format_exc())

        elif urlparse(entry.url).netloc == "gitlab.com":
            print("%s is a Gitlab project" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
            }
            print("Project Data:")
            print(project_data)
            writer_gitlab_projects.writerow(project_data)

        else:
            print("%s is hosted on custom platform" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
            }

            print("Project Data:")
            print(project_data)
            writer_custom_platform_projects.writerow(project_data)

csv_github_projects.close()
csv_github_organizations.close()
csv_gitlab_projects.close()
csv_custom_platform_projects.close()

------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
pvlib-python is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 3786
pvlib-python is an active project
Code of conduct found
Contribution guide found
Last Update
Repository(full_name="pvlib/pvlib-python")
Organization Data:
{'organization_name': 'pvlib', 'organization_user_name': 'pvlib', 'organization_url': 'https://github.com/pvlib', 'organization_avatar': 'https://avatars.githubusercontent.com/u/11037261?v=4', 'organization_location': None, 'organization_public_repos': 4, 'organization_created': '2015/02/17, 00:04:09', 'organization_last_update': '2021/11/18, 06:27:37'}
Project Data:
{'project_name': 'pvlib-python', 'git_namespace': 'pvlib', 'git_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A set of documented functions for simulating the performance of photovoltaic energy systems.', 'topics': 'solar-energy,python,renew

{'project_name': 'rdtools', 'git_namespace': 'NREL', 'git_url': 'https://github.com/NREL/rdtools.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems.', 'topics': '', 'organization': 'National Renewable Energy Laboratory', 'organization_user_name': 'NREL', 'created': '2016/11/18, 22:17:01', 'last_update': '2021/09/17, 23:38:31', 'total_commits_last_year': 15, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/09/17, 23:38:32', 'last_released_date': '2021/09/17, 23:43:12', 'last_release_tag_name': '2.1.0', 'total_number_of_commits': 558, 'community_development_distribution_score': 0.46981627296587924, 'stargazers_count': 99, 'dominating_language': 'Python', 'languages': 'Python', 'homepage': 'https://rdtools.readthedocs.io/', 'closed_issues': 267, 'open_issues': 31, 'closed_pullrequests': 175, 'open_pullrequests': 3, 'good_first_issue': 0, 'license'

{'project_name': 'photovoltaic', 'git_namespace': 'pvedu', 'git_url': 'https://github.com/pvedu/photovoltaic.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A Python library used in photovoltaics.', 'topics': '', 'organization': 'oemof community', 'organization_user_name': None, 'created': '2017/08/29, 23:41:59', 'last_update': '2021/06/15, 05:41:11', 'total_commits_last_year': 0, 'project_active_checkbox': 'false', 'last_issue_closed': '2018/01/11, 18:59:48', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 51, 'community_development_distribution_score': 0.26530612244897955, 'stargazers_count': 37, 'dominating_language': 'HTML', 'languages': 'HTML,Python,Batchfile', 'homepage': None, 'closed_issues': 1, 'open_issues': 2, 'closed_pullrequests': 1, 'open_pullrequests': 1, 'good_first_issue': 0, 'license': 'GPL-3.0', 'contributors': 3, 'accepts_donations': 'false', 'donation_platforms': None, 'code_of_conduct': 'false', 'contribution_guide'

Custom license found
No Release found
list index out of range
Last Update
Repository(full_name="tdcosim/TDcoSim")
Organization Data:
{'organization_name': 'TDcoSim Team', 'organization_user_name': 'tdcosim', 'organization_url': 'https://github.com/tdcosim', 'organization_avatar': 'https://avatars.githubusercontent.com/u/52003368?v=4', 'organization_location': 'Chicago', 'organization_public_repos': 2, 'organization_created': '2019/06/19, 14:31:04', 'organization_last_update': '2021/10/16, 03:43:40'}
Project Data:
{'project_name': 'SolarPV-DER-simulation-utility', 'git_namespace': 'tdcosim', 'git_url': 'https://github.com/tdcosim/SolarPV-DER-simulation-tool.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Allows user to run dynamics simulations for solar photovoltaic distributed energy resource connected to a stiff voltage source or to an external program.', 'topics': '', 'organization': 'TDcoSim Team', 'organization_user_name': 'tdcosim', 'created': '2019/03/19, 21:59:40'

{'project_name': 'solcore5', 'git_namespace': 'qpv-research-group', 'git_url': 'https://github.com/qpv-research-group/solcore5.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A multi-scale, Python-based library for the modeling of solar cells and semiconductor materials.', 'topics': 'photovoltaic,semiconductor,solar-cells,python,hacktoberfest', 'organization': 'Quantum Photovoltaics Research Group', 'organization_user_name': 'qpv-research-group', 'created': '2017/10/29, 20:13:23', 'last_update': '2021/11/03, 05:32:48', 'total_commits_last_year': 45, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/24, 09:49:58', 'last_released_date': '2021/09/18, 20:35:46', 'last_release_tag_name': 'v5.7.5', 'total_number_of_commits': 742, 'community_development_distribution_score': 0.4753199268738574, 'stargazers_count': 71, 'dominating_language': 'Python', 'languages': 'Python,Fortran', 'homepage': 'https://www.solcore.solar/', 'closed_issues': 163, 'open_issues': 44, '

...

{'project_name': 'Open Source Python Packages in Hydrology', 'git_namespace': 'raoulcollenteur', 'git_url': 'https://github.com/raoulcollenteur/Python-Hydrology-Tools.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'Holds a list of open source Python packages interesting to Hydrologists.', 'topics': 'hydrology,open-source,python,hydrologists', 'organization': 'Stripe', 'organization_user_name': None, 'created': '2018/07/06, 16:48:26', 'last_update': '2021/11/16, 16:43:23', 'total_commits_last_year': 8, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/05/17, 19:00:24', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 66, 'community_development_distribution_score': 0.04761904761904767, 'stargazers_count': 189, 'dominating_language': '', 'languages': '', 'homepage': '', 'closed_issues': 3, 'open_issues': 1, 'closed_pullrequests': 3, 'open_pullrequests': 0, 'good_first_issue': 0, 'license': 'CC0-1.0', 'contributors': 4, 'accepts_don

{'project_name': 'Awesome Sustainability Jobs', 'git_namespace': 'pogopaule', 'git_url': 'https://github.com/pogopaule/awesome-sustainability-jobs.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'A curated list of companies in the sustainability sector that have jobs for devs.', 'topics': 'awesome-list,sustainability,jobs,jobsearch,job-search,renewable-energies,renewables,circular-economy,emobility,job-portal,jobseeker,job-board,carbon-footprint,climate-change,climatechange,green-tech,climate-action', 'organization': 'owid', 'organization_user_name': None, 'created': '2020/11/22, 21:00:23', 'last_update': '2021/11/15, 19:50:42', 'total_commits_last_year': 353, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/05, 09:48:19', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 355, 'community_development_distribution_score': 0.02564102564102566, 'stargazers_count': 129, 'dominating_language': 'Python', 'languages': 'Python,HTML', '

{'project_name': 'giscoR', 'git_namespace': 'rOpenGov', 'git_url': 'https://github.com/rOpenGov/giscoR.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'An R API package that helps to retrieve data from Eurostat Geographic Information System of the Commission.', 'topics': 'r,gisco,eurostat-data,api-wrapper,eurostat,cran,thematic-maps,r-package,rstats,spatial,ggplot2,ropengov', 'organization': 'rOpenGov', 'organization_user_name': 'rOpenGov', 'created': '2020/09/23, 19:40:44', 'last_update': '2021/11/15, 12:55:09', 'total_commits_last_year': 361, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/28, 19:59:51', 'last_released_date': '2021/10/06, 12:04:21', 'last_release_tag_name': 'v0.3.1', 'total_number_of_commits': 632, 'community_development_distribution_score': 0.154485049833887, 'stargazers_count': 29, 'dominating_language': 'R', 'languages': 'R', 'homepage': 'https://ropengov.github.io/giscoR/', 'closed_issues': 42, 'open_issues': 1, 'closed_pullrequests':

URL: https://github.com/mysociety/caps
Climate Action Plans is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 3702
Climate Action Plans is an active project
No license information found
No Release found
list index out of range
Last Update
Repository(full_name="mysociety/retirement")
Organization Data:
{'organization_name': 'mySociety', 'organization_user_name': 'mysociety', 'organization_url': 'https://github.com/mysociety', 'organization_avatar': 'https://avatars.githubusercontent.com/u/141338?v=4', 'organization_location': 'UK', 'organization_public_repos': 289, 'organization_created': '2009/10/18, 17:02:17', 'organization_last_update': '2021/11/19, 13:46:28'}
Project Data:
{'project_name': 'Climate Action Plans', 'git_namespace': 'mysociety', 'git_url': 'https://github.com/mysociety/caps.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'A simple, open database of local government climate action plan documents and emissions data.', 'topics': 'climate,climate-ch

{'project_name': 'AI for Earth Data Sets', 'git_namespace': 'microsoft', 'git_url': 'https://github.com/microsoft/AIforEarthDataSets.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'Notebooks and documentation for AI-for-Earth-managed datasets on Azure.', 'topics': 'aiforearth', 'organization': 'microsoft', 'organization_user_name': 'Microsoft', 'created': '2021/02/18, 16:01:53', 'last_update': '2021/11/16, 18:59:29', 'total_commits_last_year': 241, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/30, 17:03:53', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 271, 'community_development_distribution_score': 0.07659574468085106, 'stargazers_count': 142, 'dominating_language': 'Jupyter Notebook', 'languages': 'Jupyter Notebook,Python', 'homepage': '', 'closed_issues': 21, 'open_issues': 1, 'closed_pullrequests': 20, 'open_pullrequests': 0, 'good_first_issue': 0, 'license': 'MIT', 'contributors': 10, 'accepts_donations': 'false